# Loan Assistant Neural Core Workflow

## 1. Imports

In [1]:
import sys
from pathlib import Path
import torch
from models.arena import Arena
from models.dataset import DataFrameDataset
from models.neural_network import BaseLoanNN
from torch.optim import Adam
import torch.nn as nn
from logging import getLogger
from tools.log_controller import LogController

/home/emdablju/Documents/university/SUML/loan_assistant/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Setup Logging

In [2]:
log = getLogger(__name__)

log_controller = LogController(config_path=Path('config/logging_config.json'), dirs=Path('logs'))
log_controller.start()

log.info("Welcome to Loan Assistant Neural Core!")

2025-11-07 15:09:24,161 - INFO - __main__:6 - Welcome to Loan Assistant Neural Core!


## 3. Load Dataset

In [3]:
dataset_path = Path('repo/loan_data.csv')
log.info(f"Loading dataset from: {dataset_path}")
dataset = DataFrameDataset.from_csv(dataset_path, target_column='loan_status')
log.info("Dataset loaded successfully.")

2025-11-07 15:09:24,173 - INFO - __main__:2 - Loading dataset from: repo/loan_data.csv
2025-11-07 15:09:24,380 - INFO - __main__:4 - Dataset loaded successfully.


## 4. Hyperparameter Tuning

In [ ]:
# log.info("Starting hyperparameter tuning...")
def objective(trial):
        input_size = len(dataset.feature_columns)
        num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
        
        hidden_layers = nn.ModuleList()
        last_dim = input_size
        for i in range(num_hidden_layers):
            hidden_dim = trial.suggest_int(f'hidden_dim_{i}', 32, 128)
            hidden_layers.append(nn.Linear(last_dim, hidden_dim))
            last_dim = hidden_dim
        
        lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
        epochs = trial.suggest_int('epochs', 10, 50)
        
        model = BaseLoanNN(input_size=input_size, hidden_layers=hidden_layers, output_size=1)
        optimizer = Adam(model.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()

        arena = Arena(model, optimizer, criterion, dataset, enable_validation=True)
        arena.train(epochs=epochs, batch_size=32)
        
        return arena.average_val_bce
    
# best_params, best_value = Arena.tune_hyperparameters(
#     objective=objective,
#     n_trials=3
# )
# log.info("Hyperparameter tuning finished.")

2025-11-07 15:09:24,392 - INFO - __main__:1 - Starting hyperparameter tuning...


[I 2025-11-07 15:09:24,394] A new study created in memory with name: no-name-21f0b8c7-3b82-48e9-92fc-ab0c8620fa0a


2025-11-07 15:09:25,661 - INFO - models.arena:15 - Using device: cpu
2025-11-07 15:09:26,339 - INFO - models.arena:55 - Epoch [1/43], BCE: 0.2943
2025-11-07 15:09:26,386 - INFO - models.arena:68 - Validation BCE: 0.2810
2025-11-07 15:09:26,795 - INFO - models.arena:55 - Epoch [2/43], BCE: 0.2731
2025-11-07 15:09:26,831 - INFO - models.arena:68 - Validation BCE: 0.2795
2025-11-07 15:09:27,214 - INFO - models.arena:55 - Epoch [3/43], BCE: 0.2693
2025-11-07 15:09:27,246 - INFO - models.arena:68 - Validation BCE: 0.2816
2025-11-07 15:09:27,642 - INFO - models.arena:55 - Epoch [4/43], BCE: 0.2660
2025-11-07 15:09:27,671 - INFO - models.arena:68 - Validation BCE: 0.2727
2025-11-07 15:09:28,083 - INFO - models.arena:55 - Epoch [5/43], BCE: 0.2673
2025-11-07 15:09:28,119 - INFO - models.arena:68 - Validation BCE: 0.2609
2025-11-07 15:09:28,504 - INFO - models.arena:55 - Epoch [6/43], BCE: 0.2647
2025-11-07 15:09:28,536 - INFO - models.arena:68 - Validation BCE: 0.2599
2025-11-07 15:09:28,905 -

[I 2025-11-07 15:09:43,911] Trial 0 finished with value: 0.2572674085572362 and parameters: {'num_hidden_layers': 1, 'hidden_dim_0': 53, 'lr': 0.04019380658166214, 'epochs': 43}. Best is trial 0 with value: 0.2572674085572362.


2025-11-07 15:09:43,915 - INFO - models.arena:15 - Using device: cpu
2025-11-07 15:09:44,959 - INFO - models.arena:55 - Epoch [1/41], BCE: 0.3108
2025-11-07 15:09:45,003 - INFO - models.arena:68 - Validation BCE: 0.2810
2025-11-07 15:09:45,811 - INFO - models.arena:55 - Epoch [2/41], BCE: 0.2752
2025-11-07 15:09:45,850 - INFO - models.arena:68 - Validation BCE: 0.2862
2025-11-07 15:09:46,997 - INFO - models.arena:55 - Epoch [3/41], BCE: 0.2643
2025-11-07 15:09:47,071 - INFO - models.arena:68 - Validation BCE: 0.2748
2025-11-07 15:09:48,195 - INFO - models.arena:55 - Epoch [4/41], BCE: 0.2594
2025-11-07 15:09:48,250 - INFO - models.arena:68 - Validation BCE: 0.2564
2025-11-07 15:09:49,288 - INFO - models.arena:55 - Epoch [5/41], BCE: 0.2563
2025-11-07 15:09:49,337 - INFO - models.arena:68 - Validation BCE: 0.2600
2025-11-07 15:09:50,337 - INFO - models.arena:55 - Epoch [6/41], BCE: 0.2554
2025-11-07 15:09:50,388 - INFO - models.arena:68 - Validation BCE: 0.2589
2025-11-07 15:09:51,351 -

[I 2025-11-07 15:10:24,630] Trial 1 finished with value: 0.3626035652018902 and parameters: {'num_hidden_layers': 5, 'hidden_dim_0': 112, 'hidden_dim_1': 37, 'hidden_dim_2': 117, 'hidden_dim_3': 68, 'hidden_dim_4': 126, 'lr': 0.01292349572930699, 'epochs': 41}. Best is trial 0 with value: 0.2572674085572362.


2025-11-07 15:10:24,636 - INFO - models.arena:15 - Using device: cpu
2025-11-07 15:10:25,529 - INFO - models.arena:55 - Epoch [1/28], BCE: 0.3093
2025-11-07 15:10:25,573 - INFO - models.arena:68 - Validation BCE: 0.2691
2025-11-07 15:10:26,372 - INFO - models.arena:55 - Epoch [2/28], BCE: 0.2613
2025-11-07 15:10:26,415 - INFO - models.arena:68 - Validation BCE: 0.2645
2025-11-07 15:10:27,309 - INFO - models.arena:55 - Epoch [3/28], BCE: 0.2557
2025-11-07 15:10:27,353 - INFO - models.arena:68 - Validation BCE: 0.2576
2025-11-07 15:10:28,202 - INFO - models.arena:55 - Epoch [4/28], BCE: 0.2508
2025-11-07 15:10:28,244 - INFO - models.arena:68 - Validation BCE: 0.2734
2025-11-07 15:10:29,108 - INFO - models.arena:55 - Epoch [5/28], BCE: 0.2501
2025-11-07 15:10:29,152 - INFO - models.arena:68 - Validation BCE: 0.2666
2025-11-07 15:10:30,025 - INFO - models.arena:55 - Epoch [6/28], BCE: 0.2465
2025-11-07 15:10:30,067 - INFO - models.arena:68 - Validation BCE: 0.2558
2025-11-07 15:10:30,943 -

[I 2025-11-07 15:10:50,166] Trial 2 finished with value: 0.26327635727556686 and parameters: {'num_hidden_layers': 5, 'hidden_dim_0': 121, 'hidden_dim_1': 82, 'hidden_dim_2': 56, 'hidden_dim_3': 60, 'hidden_dim_4': 64, 'lr': 0.004447744934798348, 'epochs': 28}. Best is trial 0 with value: 0.2572674085572362.


2025-11-07 15:10:50,166 - INFO - models.arena:92 - Best trial:
2025-11-07 15:10:50,167 - INFO - models.arena:94 -   Value: 0.2572674085572362
2025-11-07 15:10:50,168 - INFO - models.arena:95 -   Params: 
2025-11-07 15:10:50,169 - INFO - models.arena:97 -     num_hidden_layers: 1
2025-11-07 15:10:50,170 - INFO - models.arena:97 -     hidden_dim_0: 53
2025-11-07 15:10:50,171 - INFO - models.arena:97 -     lr: 0.04019380658166214
2025-11-07 15:10:50,172 - INFO - models.arena:97 -     epochs: 43
2025-11-07 15:10:50,173 - INFO - __main__:29 - Hyperparameter tuning finished.


## 5. Initialize Model with Best Hyperparameters

In [14]:
log.info("Initializing model with best hyperparameters...")
input_size = len(dataset.feature_columns)
    
model = BaseLoanNN(input_size=13, hidden_layers=nn.ModuleList([nn.Linear(13, 64)]), output_size=1)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
log.info("Model initialized.")

2025-11-07 15:16:59,371 - INFO - __main__:1 - Initializing model with best hyperparameters...
2025-11-07 15:16:59,374 - INFO - __main__:7 - Model initialized.


## 6. Train the Final Model

In [15]:
arena = Arena(model, optimizer, criterion, dataset, enable_validation=True)

log.info("Starting final training with best hyperparameters...")
arena.train(epochs=40, batch_size=32)
log.info("Training finished.")

2025-11-07 15:17:01,667 - INFO - models.arena:15 - Using device: cpu
2025-11-07 15:17:02,086 - INFO - __main__:3 - Starting final training with best hyperparameters...
2025-11-07 15:17:02,528 - INFO - models.arena:55 - Epoch [1/40], BCE: 0.4039
2025-11-07 15:17:02,566 - INFO - models.arena:68 - Validation BCE: 0.3016
2025-11-07 15:17:03,023 - INFO - models.arena:55 - Epoch [2/40], BCE: 0.2747
2025-11-07 15:17:03,055 - INFO - models.arena:68 - Validation BCE: 0.2730
2025-11-07 15:17:03,474 - INFO - models.arena:55 - Epoch [3/40], BCE: 0.2601
2025-11-07 15:17:03,506 - INFO - models.arena:68 - Validation BCE: 0.2731
2025-11-07 15:17:03,907 - INFO - models.arena:55 - Epoch [4/40], BCE: 0.2553
2025-11-07 15:17:03,939 - INFO - models.arena:68 - Validation BCE: 0.2648
2025-11-07 15:17:04,342 - INFO - models.arena:55 - Epoch [5/40], BCE: 0.2531
2025-11-07 15:17:04,375 - INFO - models.arena:68 - Validation BCE: 0.2623
2025-11-07 15:17:04,803 - INFO - models.arena:55 - Epoch [6/40], BCE: 0.2507


## 7. Evaluate the Model

In [7]:
log.info("Starting evaluation...")
arena.evaluate(batch_size=32)
log.info("Evaluation finished.")

2025-11-07 15:11:07,370 - INFO - __main__:1 - Starting evaluation...
2025-11-07 15:11:07,427 - INFO - __main__:3 - Evaluation finished.


## 8. Save Results

In [8]:
report_path = Path('logs/report.txt')
log.info(f"Saving report to: {report_path}")
with open(report_path, 'a') as f:
    f.write("\n=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    f.write("Best Hyperparameters:\n")
    for key, value in best_params.items():
        f.write(f"  {key}: {value}\n")
    f.write(f"Best Validation BCE: {best_value:.4f}\n\n")
    f.write("Final Evaluation Results:\n")
    f.write(arena.results)
    f.write("\n=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    
log.info(f"Report saved successfully.")

2025-11-07 15:11:07,441 - INFO - __main__:2 - Saving report to: logs/report.txt
2025-11-07 15:11:07,443 - INFO - __main__:13 - Report saved successfully.


# 9. Save model

In [17]:
model_path = Path('final_model.pth')
log.info(f"Saving final model to: {model_path}")
torch.save(model.state_dict(), model_path)
log.info("Model saved successfully.")

2025-11-07 15:18:56,949 - INFO - __main__:2 - Saving final model to: final_model.pth
2025-11-07 15:18:56,965 - INFO - __main__:4 - Model saved successfully.


# 10. Load model

In [24]:
# 10. Load model
model = BaseLoanNN(input_size=13, hidden_layers=nn.ModuleList([nn.Linear(13, 64)]), output_size=1)
model.load_state_dict(torch.load('model.pth'))  

<All keys matched successfully>

In [25]:
predictions = model(dataset.features[:5])
correct = dataset.target[:5]

In [29]:
# Apply sigmoid to get probabilities
probabilities = torch.sigmoid(predictions)
print("Predicted % probabilities for the first 5 samples:")
print(probabilities)
print("Correct labels for the first 5 samples:")
print(correct)

Predicted % probabilities for the first 5 samples:
tensor([[1.4308e-07],
        [3.6891e-01],
        [7.1944e-01],
        [7.6919e-01],
        [2.0563e-01]], grad_fn=<SigmoidBackward0>)
Correct labels for the first 5 samples:
tensor([0., 1., 1., 1., 0.])
